In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("iris.csv")
data = df.drop(["Id"], axis=1)
X = data.drop(["Species"], axis=1)
y = data["Species"]
X = X.values
y = y.values
Y = pd.get_dummies(y, dtype = int)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)  # X là mảng numpy hoặc pandas
Y = Y.values
X, Y = shuffle(X_scaled, Y, random_state=42)

In [2]:
input_layer = 4
hidder_layer = 20 #chỉnh từ 10 tới 20
output_layer = 3

In [3]:
def sigmoid(x):
    x = np.clip(x, -500, 500)
    return 1 / (1 + np.exp(-x))
def sigmoid_derivative(x):
    x = np.clip(x, 1e-6, 1 - 1e-6)
    return x * (1 - x)

In [4]:
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # chống overflow
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

In [7]:
def intitial_W(input_layer, hidder_layer, output_layer):
    W_1 = np.random.rand(hidder_layer, input_layer)
    W_2 = np.random.rand(output_layer, hidder_layer)
    b_1 = np.random.rand(hidder_layer)
    b_2 = np.random.rand(output_layer)
    return W_1, W_2, b_1, b_2

In [6]:

def forward_propagation(X, W_1, W_2, b_1, b_2):
    Z1 = np.dot(X, W_1.T) + b_1
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, W_2.T) + b_2
    A = softmax(Z2)
    cache = [Z1, A1, Z2, A]
    return cache

def loss_function(Y, A):
    epsilon = 1e-12  # tránh log(0)
    A = np.clip(A, epsilon, 1 - epsilon)  # bảo vệ khỏi log(0)
    return -np.mean(np.sum(Y * np.log(A), axis=1))


def backward_propagation(A, Y, X, cache, param):
    m = A.shape[0]
    W_1, W_2, b_1, b_2 = param
    Z1, A1, Z2, A = cache
    # dA = loss_devarite(Y, A)  #(4,1)
    dZ2 = A - Y
    dW_2 = (1/m)*np.dot(dZ2.T, A1)
    db_2 = (1/m)*np.sum(dZ2, axis=0, keepdims=True)
    dA1 = np.dot(dZ2, W_2)
    dZ1 = dA1 * sigmoid_derivative(A1)
    dW_1 = (1/m)*np.dot(dZ1.T, X)
    db_1 = (1/m)*np.sum(dZ1, axis=0, keepdims=True)
    gra = [dW_1, db_1, dW_2, db_2]
    return gra

def update_param(W_1, W_2, b_1, b_2, grad, learning_rate = 0.3):
    W_1 = W_1 - learning_rate * grad[0]
    W_2 = W_2 - learning_rate * grad[2]
    b_1 = b_1 - learning_rate * grad[1]
    b_2 = b_2 - learning_rate * grad[3]
    return W_1, W_2, b_1, b_2

In [7]:
# epoch = 10000
# W_1, W_2, b_1, b_2 = intitial_W() #khởi tạo 1 lần xong nhớ cmt lại để train 100000 epoch ok
# for i in range(epoch):
#     cache = forward_propagation(X_train, W_1, W_2, b_1, b_2)
#     loss = loss_function(Y_train,cache[3])
#     param = [W_1, W_2, b_1, b_2]
#     gradient = backward_propagation(cache[3],Y_train, X_train, cache, param)
#     W_1, W_2, b_1, b_2 = update_param(W_1, W_2, b_1, b_2, gradient,0.05)
#     print(loss)

In [17]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, accuracy_score
import matplotlib.pyplot as plt
import numpy as np


def do(X, Y, lns, epoch):
    accuracies = []
    precisions = []
    recalls = []
    for hidder_layer in range(10,21):
        for ln in lns:
            print("==================================================================================")
            print(f"Kết quả huấn luyệt với hidden layer có {hidder_layer} lớp và learning rate {ln}")
            for fold in range(5):
                X_val = X[fold*30 : fold*30 + 30, :]
                Y_val = Y[fold*30 : fold*30 + 30, :]
                X_train = np.concatenate((X[:fold*30, :], X[fold*30 + 30:, :]), axis=0)
                Y_train = np.concatenate((Y[:fold*30, :], Y[fold*30 + 30:, :]), axis=0)

                W_1, W_2, b_1, b_2 = intitial_W(input_layer, hidder_layer, output_layer)

                for e in range(epoch):
                    cache = forward_propagation(X_train, W_1, W_2, b_1, b_2)
                    loss = loss_function(Y_train, cache[3])

                    param = [W_1, W_2, b_1, b_2]
                    gradient = backward_propagation(cache[3], Y_train, X_train, cache, param)

                    W_1, W_2, b_1, b_2 = update_param(W_1, W_2, b_1, b_2, gradient, ln)

                # Dự đoán trên tập validation
                cache_val = forward_propagation(X_val, W_1, W_2, b_1, b_2)
                y_pred = np.argmax(cache_val[3], axis=1)  # Dự đoán lớp có xác suất cao nhất
                y_true = np.argmax(Y_val, axis=1)        # Lớp thực tế

                # Tính ma trận nhầm lẫn
                # cm = confusion_matrix(y_true, y_pred)
                # disp = ConfusionMatrixDisplay(confusion_matrix=cm)
                # disp.plot(cmap=plt.cm.Blues)
                # plt.title(f'Confusion Matrix - Fold {fold+1}')
                # plt.show()

                # Tính Accuracy, Precision, Recall
                accuracy = accuracy_score(y_true, y_pred)
                precision = precision_score(y_true, y_pred, average='weighted')  # Tính trung bình có trọng số
                recall = recall_score(y_true, y_pred, average='weighted')        # Tính trung bình có trọng số

                # Lưu kết quả cho fold này
                accuracies.append(accuracy)
                precisions.append(precision)
                recalls.append(recall)

                # print(f"Fold {fold+1}:")
                # print(f"Accuracy: {accuracy:.4f}")
                # print(f"Precision: {precision:.4f}")
                # print(f"Recall: {recall:.4f}")
                # print("-" * 50)

            # Tính trung bình của tất cả các fold
            mean_accuracy = np.mean(accuracies)
            mean_precision = np.mean(precisions)
            mean_recall = np.mean(recalls)

            print("Average metrics across 5 folds:")
            print(f"Average Accuracy: {mean_accuracy:.4f}")
            print(f"Average Precision: {mean_precision:.4f}")
            print(f"Average Recall: {mean_recall:.4f}")


In [ ]:
lns = [0.05, 0.1, 0.5]
epoch = 10000
do(X, Y, lns, epoch)

Kết quả huấn luyệt với hidden layer có 10 lớp và learning rate 0.05


array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2]])